# Chapter02 텍스트 마이닝 첫걸음

- 이장의 핵심개념
    - 웹 크롤링으로 데이터를 수집한다
    - 키워드 추출의 방법을 알아본다.
    - 키워드 간의 연관 관계를 분석한다.
    - 텍스트 분석 결과를 시각화 한다.
    
### 2.1 웹 크롤링으로 기초 데이터 수집하기

이번 절에서는 '나무위키 최근 변경 페이지'의 텍스트 데이터르 웹 크롤링 Web Crawling 으로 수집한 다음, 데이터 내에서 등장한 키워드의 출현 빈도를 분석해 보겠다. 이를 통해 우리는 나무위키 페이지에서 현재 가장 '핫한'키워드가 무엇인지 분석할 수 있다. 웹 크롤링 혹은 웹 스크래핑 Web Scraping은 인터넷에 있는 웹페이지를 방문해서 페이지의 자료를 자동으로 수집하는 작업을 의미한다. 여기에선 파이썬으로 웹 크롤링을 진행하겠다.

#### 대상 페이지의 구조 살펴보기

크롤링을 위한 첫 번째 단계는 인터넷 익스플로러, 크롬 등의 웹브라우저를 실행하여 크롤링의 대상이 될 페이지 구조를 살펴보는 것이다. 
- 먼저 웹 브라우저의 '개발자 도구'를 실행한다. 단축키 Ctrl + Shift + l
- 리스트의 URL 정보를 수집

#### 웹 크롤링 라이브러리 사용하기

파이썬에서는 BeautifulSoup 과 requests라는 라이브러리로 웹 크롤러를 만들 수 있다. requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행하고, BeautifulSoup 모듈은 HTML 문서에서 데이터를 추출하는 작업을 수행한다. 이 모듈들을 사용하기에 앞서, 터미널(cmd) 혹은 아나콘다 프롬프트를 실행하여 아래의 세 가지 파이썬 모듈을 설치해야한다.


In [71]:
#!pip3 install lxml beautifulsoup4 requests
# !pip3 install lxml
# !pip3 install bs4
# !pip3 install html5lib

In [107]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup
import re
import lxml

다음 코드에서는 requests.get()함수로 URL의 HTML 문서를 가져온 뒤, 이를 BeautifulSOup()클래스의 soup 객체로 변환한다. 그리고 find(), find_all() 함수를 사용하여 특정 HTML 태그 혹은 특정 HTML 클래스를 가진 데이터를 가져온다.

- 페이지 리스트 가져오기

In [108]:

# 크롤링할 사이트 주소를 정의한다
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 HTML 구조에 기반하여 크롤링을 수행한다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'html.parser')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

# a 태그의 href 속성을 리스트로 추출하여 크롤링할 페이지 리스트를 생성한다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        page_urls.append(page_url)
        
# 중복 url을 제거한다.
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%EB%A7%88%EA%B8%B0%EC%95%84%20%EB%A0%88%EC%BD%94%EB%93%9C%20%EB%A7%88%EB%B2%95%EC%86%8C%EB%85%80%20%EB%A7%88%EB%8F%84%EC%B9%B4%E2%98%86%EB%A7%88%EA%B8%B0%EC%B9%B4%20%EC%99%B8%EC%A0%84/%EB%A9%94%EB%AA%A8%EB%A6%AC%EC%95%84/2021%EB%85%84
https://namu.wiki/w/%ED%8B%80:%ED%94%84%EB%9E%91%EC%8A%A4%20%EC%99%95%EA%B5%AD%EC%9D%98%20%ED%96%89%EC%A0%95%20%EA%B5%AC%EC%97%AD%EA%B3%BC%20%EA%B5%B0%EA%B5%AC
https://namu.wiki/w/%EC%98%A4%EC%85%80
https://namu.wiki/w/%EC%95%85%EB%8B%B9%EC%98%81%EC%95%A0%20%EA%B8%B8%EB%93%A4%EC%9D%B4%EA%B8%B0
https://namu.wiki/w/%EC%95%8C%EB%9E%AD%20%EB%93%A4%EB%A1%B1


위의 코드는 개발자 도구로 살펴본 HTML 구조에 기반하여 table -> tbody -> tr -> td -> a 태그 순의 HTML 계층 구조를 좁혀나가는 과정이다. 이과정을 통해 목표 태그에 도달했을 때, get(href) 함수로 태그의 속성 정보를 추출한다. get()함수는 해당 태그가 가지고 있는 특정한 속성을 추출한다.

### 텍스트 정보 수집하기

이전과 마찬가지로 개발자 도구의 마우스 포인터 모양 아이콘을 클릭한다. 그리고 문서의 '제목'부분, 문서의 '카테고리'부분, 그리고 '본문'부분을 클릭하여 HTML의 구조를 파악한다. 문서의 전체 내용은 'article'이라는 태그 안에 구성되어 있다. 그리고 제목은 'h1'태그, 카테고리 부분은 'ul' 태그 영역 안에 존재하며, 본문의 내용은 'wiki-pararaph'라는 클래스로 구성된 'div'태그 안에 위치하고 있다. 다음 코드는 최근 변경된 문서 중 한 페이지의 텍스트 정보를 크롤링한 것이다. 이전 단계와 다른 점은 get()함수 대신 text()함수를 사용하여 태그의 텍스트 정보만을 추출했다는 점이다.

- URL 페이지 정보를 기반으로 크롤링하기

In [109]:
# 하나의 최근 변경된 문서를 크롤링 한다.
req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'html.parser')
contents_table = soup.find(name="article")
title = contents_table.find_all('h1')[0]
category = contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
content_corpus_list = []

# 크롤링한 문서 정보 출력

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus = "".join(content_corpus_list)
    
print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

마기아 레코드 마법소녀 마도카☆마기카 외전/메모리아/2021년 


마기아 레코드


하위 문서  [ 펼치기ㆍ접기 ]마기☆레포ㆍ미러즈ㆍ시스템ㆍ아이템ㆍ정신강화ㆍ카미하마 시ㆍ평가가챠 ('17년ㆍ'18년ㆍ'19년ㆍ'20년ㆍ'21년)이벤트 ('17년ㆍ'18년ㆍ'19년ㆍ'20년ㆍ'21년ㆍ감정전ㆍ캠페인ㆍ특훈)메모리아 (★1ㆍ★2ㆍ★3ㆍ★4ㆍ'21년)애니메이션 (1기ㆍ2기ㆍ에피소드ㆍ엔드 카드ㆍ평가)등장인물  [ 펼치기ㆍ접기 ]★4토우카ㆍ모모코ㆍ카린ㆍ쥬리ㆍ시즈카ㆍ하구무ㆍ츠루노·펠리시아ㆍ쿄코ㆍ쿄코(수영복)ㆍ히미카ㆍ료코ㆍ엘리자ㆍ스즈네ㆍ츠바키ㆍ스루가ㆍ츠바사ㆍ나노하★3소문 츠루노ㆍ레이라ㆍ아이미ㆍ리카ㆍ세이라ㆍ메구루ㆍ란카ㆍ료우ㆍ카오루ㆍ라핀★2츠루노ㆍ마나카ㆍ아야메★4레나ㆍ아오ㆍ스나오ㆍ이로하(수영복)ㆍ야치요·미후유ㆍ소문 사나ㆍ레나짱ㆍ아마네 자매ㆍ레나·카에데ㆍ사야카ㆍ사야카(하레기)ㆍ코노하ㆍ마유ㆍ준ㆍ마요이★3아스카ㆍ나나카ㆍ메이유이ㆍ마리아ㆍ테마리ㆍ우미카ㆍ멜리사ㆍ치사토★2야치요ㆍ나츠키ㆍ아키라ㆍ세이카★4알리나ㆍ네무ㆍ치하루ㆍ시구레ㆍ알렉산드라ㆍ홀리 알리나ㆍ마미ㆍ홀리 마미ㆍ마미(수영복)ㆍ미토ㆍ츠무기ㆍ유우나ㆍ코노하·하즈키ㆍ마츠리ㆍ나데코★3카에데ㆍ모카ㆍ메루ㆍ치카ㆍ유마ㆍ하야테★2사나ㆍ히나노ㆍ카코ㆍ사쿠야ㆍ코노미★4카나기ㆍ히카루ㆍ모모코(시스터)ㆍ이로하짱ㆍ이로하·야치요ㆍ이로하·우이ㆍ마도카ㆍ마도카(하레기)ㆍ얼티밋 마도카ㆍ궁극 마도카 선배ㆍ호무라(수영복)ㆍ코코로ㆍ유키카ㆍ사유키ㆍ리코ㆍ만년 벚꽃ㆍ미츠네ㆍ만년 벚꽃(수영복)ㆍ타루토ㆍ파이널 타루토ㆍ시노부ㆍ페이트★3츠카사ㆍ마도카 선배ㆍ하즈키ㆍ마사라ㆍ호토리ㆍ오리코ㆍ하루카★2사사라ㆍ시즈쿠★1이로하★4미후유ㆍ우이ㆍ츠쿠요ㆍ미카게ㆍ작은 큐베ㆍ토우카·네무ㆍ카나기(뱀파이어)ㆍ호무라ㆍ호무라(안경)ㆍ나기사ㆍ나기사(밸런타인)ㆍ한나ㆍ렌ㆍ미쿠라ㆍ카나에ㆍ애슐리ㆍ이쿠미ㆍ루이ㆍ리카·렌ㆍ키리카ㆍ카즈미ㆍ리즈★3펠리시아짱ㆍ리아ㆍ아야카ㆍ리온ㆍ코르보ㆍ아리사ㆍ히타기★2펠리시아ㆍ카노코ㆍ에미리★1쿠로★4미타마ㆍ스다치ㆍ요즈루ㆍ리비아ㆍ미타마(하레기)NPC마녀ㆍ소문ㆍ감정마법소녀그룹마기우스의 날개ㆍ카미하마 마기아 유니온ㆍ토

### 2.2 나무위키 최근 변경 페이지 키워드 분석하기

이제 분석에 사용할 데이터가 준비되었으니, 본격적으로 텍스트 마이닝을 알아보자.

#### Step 1 크롤링:웹 데이터 가져오기

이전 단계와 동일한 방법으로 웹 데이터를 크롤링하자. 단, 이번에는 모든 URL의 데이터를 가져와 보자. 다음 코드를 실행하여 나무위키에서 최근 변경이 일어난 페이들의 URL을 page_urls라는 변수에 저장한다.

- BeautifulSoup을 이용해 웹 크롤링하기

In [110]:
# -*- coding:utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re


In [116]:
# 크롤링할 사이트 주소를 정의한다
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 HTML 구조에 기반하여 크롤링을 수행한다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'html.parser')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

In [117]:
# a태그의 href 속성을 리스트로 추출하여, 크로링 할 페이지 리스트를 생성하자.

page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)
            
# 중복 url을 제거 하자
page_urls = list(set(page_urls))

이제 이 주소들에 다시 한번 접급하여 문서의 본문과 제목, 그리고 카테고리에 등장하는 텍스트 데이터를 가져와 보자.
- 나무위키 최근 변경 데이터 크롤링하기

In [134]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    req = requests.get(page_url)
    html = req.content
    soup = BeautifulSoup(html, 'html.parser')
    contents_table = soup.find(name="article")
    title = contents_table.find_all('h1')[0]
    
    # 카테고리 정보가 없는 경우를 확인합니다.
    if len(contents_table.find_all('ul')) > 0:
        category = contents_table.find_all('ul')[0]
    else:
        category = None
        
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    content_corpus_list = []
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 “분류”라는 단어와 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
    
    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

In [135]:
df.head()

,title,category,content_text
0,마기아 레코드 마법소녀 마도카☆마기카 외전/메모리아/2021년,마기아 레코드,하위 문서 [ 펼치기ㆍ접기 ]마기☆레포ㆍ미러즈ㆍ시스템ㆍ아이템ㆍ정신강화ㆍ카미하마 시...
1,악당영애 길들이기,웹소설/ㅇ노벨피아/작품2021년 소설퓨전 판타지 웹소설성인 판타지 소설책빙의물나무위...,"악당영애 길들이기장르판타지, 아카데미, 책빙의,악녀, 하렘, 떡타지작가미츄리출판사노..."
2,알랭 들롱,프랑스 남배우전직 배우일드프랑스 출신 인물복수국적자1935년 출생1957년 데뷔20...,은(는) 여기로 연결됩니다. 만화 벨제바브의 등장인물에 대한 내용은 알랭들롱(...
3,작곡가,직업작곡가,1. 개요2. 상세3. 대한민국 대중음악계에서 작곡가의 위치4. 서구권과 일본 대중...
4,쿠키런: 킹덤/건물/꾸미기,쿠키런: 킹덤,상위 문서: 쿠키런: 킹덤/건물게임 요소쿠키(특별한 인연) · 보물 · 건물...


위의 실행 결과는 모든 URL의 텍스트 데이터를 가져온 뒤, 이를 데이터 프레임의 형태로 변환한 것이다. 데이터에 등장하는 불필요한 문자인 '\n','분류'는 크롤링 과정에서 replace()함수로 제거 한다.

### Step 2 추출: 키워드 정보 추출하기

다음은 수집한 텍스트 데이터에서 키워드 정보를 추출하는 단계이다. 이를 위해 **텍스트 전처리** 작업이 필요하다. 텍스트 전처리는 특수문자나 외국어를 제거하는 등의 과정을 포함한다. 그런데 이는 언어와 상황마다 조금씩 다를 수 있다. 예를 들어 스팸메일을 분류하는 텍스트 마이닝의 경우, 특수문자나 외국어가 분석의 중요한 힌트가 될 수 있기 때문에 이를 제거하지 않는 편이다. 반면, 키워드 분석처럼 '단어'를 추출하는 것이 목적이라면 특정 언어의 글자만을 추출하기도 한다.

파이썬에서는 're'라는 모듈을 통해 정규표현식을 사용할 수 있다. 정규표현식이란 특정한 규칙을 가진 문자열의 집합을 표현하는 형식이다. 만약 다음 코드와 같이 re.compile('[^ㄱㅡㅣ가ㅡ힣]+')이라는 코드로 한글에 대한 정규표현식을 정의하면 대상이 되는 텍스트 데이터에서 한글만 추출할 수 있게 된다.

- 텍스트 데이터 전처리하기


In [139]:
# 텍스트 정제 함수: 한글 이외의 문자는 전부 제거 한다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]+') #한글의 정규표현식을 나타낸다.
    result = hangul.sub('',text)
    return result
print(text_cleaning(df['content_text'][0]))

하위 문서   펼치기접기 마기레포미러즈시스템아이템정신강화카미하마 시평가가챠 년년년년년이벤트 년년년년년감정전캠페인특훈메모리아 년애니메이션 기기에피소드엔드 카드평가등장인물   펼치기접기 토우카모모코카린쥬리시즈카하구무츠루노펠리시아쿄코쿄코수영복히미카료코엘리자스즈네츠바키스루가츠바사나노하소문 츠루노레이라아이미리카세이라메구루란카료우카오루라핀츠루노마나카아야메레나아오스나오이로하수영복야치요미후유소문 사나레나짱아마네 자매레나카에데사야카사야카하레기코노하마유준마요이아스카나나카메이유이마리아테마리우미카멜리사치사토야치요나츠키아키라세이카알리나네무치하루시구레알렉산드라홀리 알리나마미홀리 마미마미수영복미토츠무기유우나코노하하즈키마츠리나데코카에데모카메루치카유마하야테사나히나노카코사쿠야코노미카나기히카루모모코시스터이로하짱이로하야치요이로하우이마도카마도카하레기얼티밋 마도카궁극 마도카 선배호무라수영복코코로유키카사유키리코만년 벚꽃미츠네만년 벚꽃수영복타루토파이널 타루토시노부페이트츠카사마도카 선배하즈키마사라호토리오리코하루카사사라시즈쿠이로하미후유우이츠쿠요미카게작은 큐베토우카네무카나기뱀파이어호무라호무라안경나기사나기사밸런타인한나렌미쿠라카나에애슐리이쿠미루이리카렌키리카카즈미리즈펠리시아짱리아아야카리온코르보아리사히타기펠리시아카노코에미리쿠로미타마스다치요즈루리비아미타마하레기마녀소문감정마법소녀그룹마기우스의 날개카미하마 마기아 유니온토키메 일족 유나네오 마기우스히메나푸엘라 케어포클로어 오브 제로애니메이션오리지널쿠로에토우카모모코카린쥬리시즈카하구무츠루노펠리시아쿄코쿄코수영복히미카료코엘리자스즈네츠바키스루가츠바사나노하소문 츠루노레이라아이미리카세이라메구루란카료우카오루라핀츠루노마나카아야메레나아오스나오이로하수영복야치요미후유소문 사나레나짱아마네 자매레나카에데사야카사야카하레기코노하마유준마요이아스카나나카메이유이마리아테마리우미카멜리사치사토야치요나츠키아키라세이카알리나네무치하루시구레알렉산드라홀리 알리나마미홀리 마미마미수영복미토츠무기유우나코노하하즈키마츠리나데코카에데모카메루치카유마하야테사나히나노카코사쿠야코노미카나기히카루모모코시스터이로하짱이로하야치요이로하우이마도카

모든 데이터에 전처리를 적용하기 위해서는 apply() 함수를 사용한다. 다음 코드는 title,category,content_text 3개의 피처에 apply()함수를 적용한 것이다. 이를 head()함수로 출력하면 한글을 제외한 문자들이 제거된 것을 확인할 수 있다.
- 모든 데이터에 전처리 적용하기

In [141]:
# 각 피처마다 데이터 전처리를 적용한다.
df['title'] = df['title'].apply(lambda x: text_cleaning(x))
df['category'] = df['category'].apply(lambda x: text_cleaning(x))
df['content_text'] = df['content_text'].apply(lambda x: text_cleaning(x))


In [142]:
df.head()

,title,category,content_text
0,마기아 레코드 마법소녀 마도카마기카 외전메모리아년,마기아 레코드,하위 문서 펼치기접기 마기레포미러즈시스템아이템정신강화카미하마 시평가가챠 년년년년...
1,악당영애 길들이기,웹소설ㅇ노벨피아작품년 소설퓨전 판타지 웹소설성인 판타지 소설책빙의물나무위키 웹소설 ...,악당영애 길들이기장르판타지 아카데미 책빙의악녀 하렘 떡타지작가미츄리출판사노벨피아연재...
2,알랭 들롱,프랑스 남배우전직 배우일드프랑스 출신 인물복수국적자년 출생년 데뷔년 은퇴,은는 여기로 연결됩니다 만화 벨제바브의 등장인물에 대한 내용은 알랭들롱벨제바브 ...
3,작곡가,직업작곡가,개요 상세 대한민국 대중음악계에서 작곡가의 위치 서구권과 일본 대중음악계에서 작곡...
4,쿠키런 킹덤건물꾸미기,쿠키런 킹덤,상위 문서 쿠키런 킹덤건물게임 요소쿠키특별한 인연 보물 건물랜드마크 꾸미...


다음 과정은 키워드를 추출한 뒤, 빈도 분석을 수행하는 과정이다. 여기서 키워드를 추출한다는 것은 무엇을 의미 할까? 키워드 추출이란 좁은 의미에서는 **명사, 혹은 형태소 단위의 문자열**을 추출하는 것이다. 이를 수행 하기 위해 **말뭉치**라는 것을 만들어야 한다. 말뭉치는 말 그대로 텍스트 데이터의 뭉텅이를 의미한다. 이번 예제에서는 제목단위, 카테고리 단위, 본문 단위의 키워드를 분석하기 위해 제목 말뭉치, 카테고리 말뭉치, 본문 말뭉치 총 3개의 말뭉치를 생성한다. 다음 코드에서는 텍스트 피처를 tolist()로 추출한 뒤, join() 함수로 말뭉치를 생성해주었다. 실행 결과는 제목 말뭉치의 출력 결과이다.
- 말뭉치 만들기

In [143]:
# 각 피처마다 말뭉치를 생성한다.
title_corpus = "".join(df['title'].tolist())
category_corpus = "".join(df['category'].tolist())
content_corpus = "".join(df['content_text'].tolist())
print(title_corpus)

마기아 레코드 마법소녀 마도카마기카 외전메모리아년 악당영애 길들이기 알랭 들롱 작곡가 쿠키런 킹덤건물꾸미기 바이에른뮌헨 마이너 갤러리 건담 아르테미스 콘돔 미국인 제대 국회의원 선거선거구 획정   타워 메르세데스벤츠 블랙 메사게임    리그년    기믹 로자바 나인로드 오브 히어로즈 동남여객  변호사제인격 불멸의 게이머 주지유 스테인 재재논란 및 사건사고 주접 한강 의대생 실종 사건 조강옥 료마전등장인물 칼슘 파일전통적 플란데런 문장위키미디어 공용 로켓단 스갤문학 모범택시드라마등장인물   알부민 구로다 기요타카 천재캐릭터 을지대학교 버스트  안성 버스  이한도 김사랑   킹갓캐슬 한국프로농구 시즌 무화과맛 쿠키   더미 데이터 여자친구아이돌    모드      국가구 소련 고양동산고등학교  프란체스코 베첼리오 모데카이저 볼드모트 크몽 이투스  오월의 청춘 틀 참가팀 메르세데스 하남 버스  두분토론 알레비파 라이덴메탈기어 시리즈 파일 강태린  레이드배틀성 측천무후 괴물스위트홈 유재석수상 내역 파일플란데런 백국 문장 쌍검몬스터 헌터 라이즈   영구결번스포츠 한화 이글스 군산복합체 실버 채리엇 레퀴엠 호랑가시나무땅   시험 기관별 진도 차이 파일 


이제 각 말뭉치 안에서 등장하는 형태소를 추출하겠다. 파이썬의 한국어 형태소 추출 라이브러리 중 가장 쉽게 사용할 수 있는 konlpy를 사용한다. 다음 코드는 konlpy의 Okt tagger를 이용하여 형태소를 추출한느 과정이다. 시행 결과 Counter 객체에 {'단어':'빈도'} 형태로 키워드가 추출된 것을 확인할 수 있다. 그런데 추출 키워드를 자세히 살펴보면 한 글자 키워드 혹은 실질적인 의미가 없는 키워드 ('분류','필요' 등의 형태소)들을 발견할 수 있다. 이런 키워드는 어떻게 처리해야할까?
- konlpy를 이용해 키워드 추출하기

In [146]:
from konlpy.tag import Okt
from collextions import Counter

ModuleNotFoundError: No module named 'collextions'

In [145]:
!pip3 install konlpy

  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-4.9.3


In [147]:
!pip3 install collextions

ERROR: Could not find a version that satisfies the requirement collextions (from versions: none)
ERROR: No matching distribution found for collextions


In [148]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [149]:
pip install JPype1-0.5.7-cp27-none-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: JPype1-0.5.7-cp27-none-win_amd64.whl is not a supported wheel on this platform.


In [150]:
 java -version

NameError: name 'java' is not defined

In [152]:
#!pip3 install --upgrade pip
!pip3 install JPype1-0.5.7-cp27-none-win_amd64.whl

ERROR: JPype1-0.5.7-cp27-none-win_amd64.whl is not a supported wheel on this platform.
